### Install pymysql 
```
$ pip install pymysql
```

In [1]:
import pymysql
import pymysql.cursors

In [24]:
"""
View connection properties 
https://pymysql.readthedocs.io/en/latest/modules/connections.html
"""
con = pymysql.connect(host = "az-demo.c33yjtahjyxw.us-west-2.rds.amazonaws.com"
                      , user = "admin"
                      , password = "masterkey"
                      , autocommit="True"
                      , charset="utf8mb4"
                      , connect_timeout = 3600
                      , cursorclass=pymysql.cursors.DictCursor
                     )
cur = con.cursor()
cur.execute("SELECT VERSION()")
version = cur.fetchone()
print("Database version: ", version)

Database version:  {'VERSION()': '8.0.15'}


In [6]:
cur.execute("SHOW DATABASES")
list(cur.fetchall())

[{'Database': 'information_schema'},
 {'Database': 'mysql'},
 {'Database': 'performance_schema'},
 {'Database': 'sys'}]

In [7]:
db_name = "stocks"
cur.execute(f"CREATE DATABASE IF NOT EXISTS {db_name} ")
cur.execute("SHOW DATABASES")
cur.fetchall()

[{'Database': 'information_schema'},
 {'Database': 'mysql'},
 {'Database': 'performance_schema'},
 {'Database': 'stocks'},
 {'Database': 'sys'}]

In [26]:
cur.execute("USE stocks")

0

In [9]:
cur.execute("DROP TABLE IF EXISTS stock")

/anaconda3/lib/python3.7/site-packages/pymysql/cursors.py:329: Warning: (1051, "Unknown table 'stocks.stock'")
  self._do_get_result()


0

In [10]:
cur.execute("""
CREATE TABLE IF NOT EXISTS stock (
    id INTEGER PRIMARY KEY AUTO_INCREMENT, 
    date DATE NOT NULL, 
    high FLOAT NOT NULL,
    low FLOAT NOT NULL,
    open FLOAT NOT NULL,
    close FLOAT NOT NULL,
    adjclose FLOAT NOT NULL,
    volume FLOAT NOT NULL,
    symbol VARCHAR(50) NOT NULL
)
""")

0

In [11]:
cur.execute("SHOW tables")
cur.fetchall()

[{'Tables_in_stocks': 'stock'}]

In [13]:
cur.execute("select * from stock")
cur.fetchall()

()

In [14]:
def csv_to_dict(file_name):
    import csv
    with open(file_name, encoding="utf-8") as f:
        rows = list(csv.DictReader(f))
        return rows
rows = csv_to_dict("/data/stocks.small.csv")
rows[:3]

[OrderedDict([('date', '2000-07-18'),
              ('open', '144.8125'),
              ('high', '144.828125'),
              ('low', '141.4375'),
              ('close', '143.0'),
              ('volume', '5.06836E7'),
              ('adjclose', '50.155473'),
              ('symbol', 'INTC')]),
 OrderedDict([('date', '2000-07-20'),
              ('open', '32.93751'),
              ('high', '34.25001'),
              ('low', '32.8125'),
              ('close', '33.75'),
              ('volume', '3288300.0'),
              ('adjclose', '8.789734'),
              ('symbol', 'BEN')]),
 OrderedDict([('date', '2000-07-24'),
              ('open', '64.25'),
              ('high', '67.312477'),
              ('low', '64.187523'),
              ('close', '64.75'),
              ('volume', '948800.0'),
              ('adjclose', '7.689567'),
              ('symbol', 'APH')])]

In [15]:
for row in rows:
    row["open"] = float(row["open"])
    row["close"] = float(row["close"])
    row["high"] = float(row["high"])
    row["low"] = float(row["low"])
    row["adjclose"] = float(row["adjclose"])
    row["volume"] = float(row["volume"])

In [16]:
insertable_rows = [tuple(row.values()) for row in rows]

In [17]:
insertable_rows[0]

('2000-07-18',
 144.8125,
 144.828125,
 141.4375,
 143.0,
 50683600.0,
 50.155473,
 'INTC')

In [18]:
insert_statement = """
INSERT INTO stock 
(date,open,close,high,low,adjclose,volume,symbol) 
VALUES 
(%s,%s,%s,%s,%s,%s,%s,%s)
""" 

In [19]:
cur.execute(insert_statement, tuple(rows[0].values()))

1

In [20]:
cur.execute("select * from stock")
cur.fetchall()

[{'id': 1,
  'date': datetime.date(2000, 7, 18),
  'high': 141.438,
  'low': 143.0,
  'open': 144.812,
  'close': 144.828,
  'adjclose': 50683600.0,
  'volume': 50.1555,
  'symbol': 'INTC'}]

In [21]:
cur.executemany(insert_statement, insertable_rows)

1846

In [22]:
cur.execute("select count(*) from stock")
cur.fetchall()

[{'count(*)': 1847}]

In [23]:
con.close()